# NLP Project

In [2]:
import findspark
findspark.init('/home/ubuntu/Spark/spark-3.3.0-bin-hadoop3')

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('NLP').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/24 09:19:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [7]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [8]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [9]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import length

In [11]:
data = data.withColumn('length', length(data['text']))

In [12]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [18]:
tokenizer = Tokenizer(inputCol='text', outputCol='tokenText')
stopRemove = StopWordsRemover(inputCol='tokenText', outputCol='stopToken')
countVec = CountVectorizer(inputCol='stopToken', outputCol='cVector')
idf = IDF(inputCol='cVector', outputCol='tfIDF')
hamSpamToNumeric = StringIndexer(inputCol='class', outputCol='label')

In [19]:
from pyspark.ml.feature import VectorAssembler

In [28]:
cleanUp = VectorAssembler(inputCols=['tfIDF', 'length'], outputCol='features')

In [29]:
from pyspark.ml.classification import NaiveBayes

In [30]:
nb = NaiveBayes()

In [31]:
from pyspark.ml import Pipeline

In [32]:
dataPrepPipe = Pipeline(stages=[hamSpamToNumeric, tokenizer, stopRemove, countVec, idf, cleanUp])

In [33]:
cleaner = dataPrepPipe.fit(data)

In [34]:
cleanData = cleaner.transform(data)

In [35]:
cleanData.columns

['class',
 'text',
 'length',
 'label',
 'tokenText',
 'stopToken',
 'cVector',
 'tfIDF',
 'features']

In [36]:
cleanData = cleanData.select('label', 'features')

In [37]:
cleanData.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [38]:
traning, test = cleanData.randomSplit([0.7, 0.3])

In [40]:
spamDetector = nb.fit(traning)

22/09/24 09:38:36 WARN DAGScheduler: Broadcasting large task binary with size 1131.2 KiB


22/09/24 09:38:38 WARN DAGScheduler: Broadcasting large task binary with size 1120.9 KiB


In [41]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [42]:
test_results = spamDetector.transform(test)

In [43]:
test_results.show()

22/09/24 09:43:18 WARN DAGScheduler: Broadcasting large task binary with size 1353.5 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-572.45967328352...|[0.99999999999965...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-998.67087466430...|[1.0,1.0619816361...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-877.22354728100...|[1.0,1.5136305431...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-542.52375451077...|[1.0,2.0688721057...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-691.45390242759...|[1.0,4.4098284381...|       0.0|
|  0.0|(13424,[0,1,24,31...|[-339.45531489089...|[1.0,7.5699172486...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1487.7070849570...|[0.99998308497899...|       0.0|
|  0.0|(13424,[0,1,416,6...|[-312.45942111299...|[0.99999999963160...|       0.0|
|  0.0|(13424,[0,1,874,1...|[-96.166791475549...|[0.99999996311074...|       0.0|
|  0.0|(13424,[0

In [44]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [46]:
acc_eval = MulticlassClassificationEvaluator()

In [47]:
acc = acc_eval.evaluate(test_results)

22/09/24 09:45:08 WARN DAGScheduler: Broadcasting large task binary with size 1358.0 KiB


In [48]:
print('ACC')
print(acc)

ACC
0.9148092581535819
